In [ ]:
import requests
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt


check_start_time = datetime(2024, 10, 23, 1, 56, 0) # UTC time
check_end_time = datetime(2024, 10, 23, 2, 16, 0)

# Define Jaeger API URL (the jager service is at the social-network's  namespace)
JAEGER_API_URL_k8s = "http://192.168.69.232:16686/api/traces"
JAEGER_API_URL_NetMARKS = "http://192.168.69.236:16686/api/traces"
JAEGER_API_URL_TraDE = "http://192.168.79.79:16686/api/traces"

JAGER_API_URLs = [JAEGER_API_URL_k8s, JAEGER_API_URL_NetMARKS, JAEGER_API_URL_TraDE]

# Define parameters for the query
service_name = "compose-post-service" #'home-timeline-service' #"compose-post-service"
lookback = "60m"
limit = 1000

# Function to query Jaeger API
def query_jaeger(service_name, lookback, limit, jaeger_url):
    params = {
        "service": service_name,
        "lookback": lookback,
        "limit": limit
    }
    response = requests.get(jaeger_url, params=params)
    response.raise_for_status()
    return response.json()


# Parse the trace data
def parse_jaeger_data(data):
    traces = data['data']
    trace_list = []
    for trace in traces:
        trace_id = trace['traceID']
        for span in trace['spans']:
            operation = span['operationName']
            start_time = datetime.utcfromtimestamp(span['startTime'] / 1e6)
            duration = span['duration'] / 1e3  # Convert microseconds to milliseconds
            trace_list.append({
                "trace_id": trace_id,
                "operation": operation,
                "start_time": start_time,
                "duration": duration
            })
    return pd.DataFrame(trace_list)


# Fetch trace data from Jaeger
data = query_jaeger(service_name, lookback, limit, JAEGER_API_URL_k8s)

# Parse the data into a DataFrame
df = parse_jaeger_data(data)

# Convert 'start_time' to datetime and 'duration' to numeric
df['start_time'] = pd.to_datetime(df['start_time'])




